# TVB‑AdEx • Basic Test Run

This notebook runs a **minimal** whole‑brain AdEx simulation using the toolbox and then loads the saved results for a quick sanity check plot.

**What it does**
1. Imports your local modules: `parameters`, `simconfig`, `run_simulations`, `utils`  
2. Lists available subjects (if helper is available) and picks one (default: `sub-01`)  
3. Builds a `SimConfig` with a short run (2 s) and runs a single seed  
4. Loads the last saved `step_*.npy` and plots raw E/I signals (if available)

> **Note:** This assumes your repository layout and CSV connectivity files are in place:
>
> ```
> data/organised/dataframes/SC_CNT.csv
> data/organised/dataframes/TL_CNT.csv
> ```
> and that your Python path includes this repository (the working directory should be the repo root).


In [1]:
%precision 2
%load_ext autoreload
%autoreload 2

import os, numpy as np, matplotlib.pyplot as plt, pathlib as Path


from parameters import Parameters, ListParameter
from simconfig import SimConfig
from run_simulations import run_simulations_parallel, run_simulation
from plotting import plot_tvb_results   
from utils import get_result           

/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [3]:
# Instantiate default parameter set
general_parameters = Parameters()

# --- Make Raw monitor ON (and optional TemporalAverage OFF initially) ---
general_parameters.parameter_monitor['Raw'] = True
general_parameters.parameter_monitor['TemporalAverage'] = False
general_parameters.parameter_monitor['Bold'] = False
general_parameters.parameter_monitor['Ca'] = False

# --- Continuous outputs in one file: set save_time = run_sim later (see next cell) ---
# We'll assign below once we define run_sim

# Model flavour (no gK_gNa by default)
general_parameters.parameter_model['gK_gNa'] = False

# Example TF polynomial coeffs (diVolo label via ListParameter)
config = 'diVolo'
P_e = [-0.04983106, 0.00506355, -0.02347012, 0.00229515, -0.00041053, 0.01054705, -0.03659253, 0.00743749, 0.00126506, -0.04072161]
P_i = [-0.05149122, 0.00400369, -0.00835201, 0.00024142, -0.00050706, 0.00143454, -0.01468669, 0.00450271, 0.00284722, -0.0153578]
general_parameters.parameter_model['P_e'] = ListParameter('P_e', P_e, config)
general_parameters.parameter_model['P_i'] = ListParameter('P_i', P_i, config)

# Seeds, duration, transients
seeds = [0, 1, 2]           # three seeds per config
run_sim = 5000.0            # ms
cut_transient = 400.0       # ms you want trimmed in *analyses* (keep small here; you can adjust)

# Make the output a single continuous file:
general_parameters.parameter_simulation['save_time'] = run_sim  # <-- key: no chunking

# Optional: integrator step (ms)
general_parameters.parameter_integrator['dt'] = 0.1


In [4]:
# Sweep b_e = 5, 10, ..., 60
b_es = list(range(5, 61, 5))

simconfigs = [
    SimConfig(
        general_parameters=general_parameters,
        run_sim=run_sim,
        cut_transient=cut_transient,
        parameters={'b_e': b_e, 'E_L_e': -63},
        # Stop if TemporalAverage monitor’s first var > 150 Hz after 1500 ms
        stop_conditions=[(1, 0, -np.inf, 150 * 1e-3, 1500)],  # (monitor_id, var_id, min, max, t_ms)
        auto_report=True
    )
    for b_e in b_es
]

print(f"Prepared {len(simconfigs)} simulation configs:")
for sc in simconfigs:
    print("  -", sc.get_plot_title())


Prepared 12 simulation configs:
  - b_e=5, E_L_e=-63
  - b_e=10, E_L_e=-63
  - b_e=15, E_L_e=-63
  - b_e=20, E_L_e=-63
  - b_e=25, E_L_e=-63
  - b_e=30, E_L_e=-63
  - b_e=35, E_L_e=-63
  - b_e=40, E_L_e=-63
  - b_e=45, E_L_e=-63
  - b_e=50, E_L_e=-63
  - b_e=55, E_L_e=-63
  - b_e=60, E_L_e=-63


In [5]:
# Where to store run folders
simu_path = "./results/synch"
if not os.path.exists(simu_path):
    os.makedirs(simu_path)

# Run all configs × seeds in parallel
_ = run_simulations_parallel(
    simconfigs,
    seeds=seeds,
    stop_early=False,
    n_tasks_concurrent=None,   # defaults to CPU-1
    save_path_root=simu_path,
    no_skip=False,             # re-use existing completed runs
    param_change_with_seed=None
)


/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)
/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)
/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)
/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)
/Users/borjan/miniconda3/envs/brainlab/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60

path =  ./results/synch/b_e_15_E_L_e_-63_2
Initialize Simulator
Setting random seed: 2
path =  ./results/synch/b_e_10_E_L_e_-63_1
Initialize Simulator
Setting random seed: 1
path =  ./results/synch/b_e_5_E_L_e_-63_2
Initialize Simulator
Setting random seed: 2
path =  ./results/synch/b_e_15_E_L_e_-63_1
Initialize Simulator
Setting random seed: 1
path =  ./results/synch/b_e_5_E_L_e_-63_1
Initialize Simulator
Setting random seed: 1
path =  ./results/synch/b_e_5_E_L_e_-63_0
Initialize Simulator
Setting random seed: 0
path =  ./results/synch/b_e_10_E_L_e_-63_2
Initialize Simulator
Setting random seed: 2
path =  ./results/synch/b_e_20_E_L_e_-63_1
Initialize Simulator
Setting random seed: 1
path =  ./results/synch/b_e_15_E_L_e_-63_0
Initialize Simulator
Setting random seed: 0
path =  ./results/synch/b_e_10_E_L_e_-63_0
Initialize Simulator
Setting random seed: 0
path =  ./results/synch/b_e_20_E_L_e_-63_0
Initialize Simulator
Setting random seed: 0
Loaded subject-specific connectivity for sub-0

In [6]:
# Sanity Check

# Pick the first config and seed 0
cfg0 = simconfigs[0]
folder_root, sim_name = cfg0.get_sim_name(seed=seeds[0])
run_dir = Path(simu_path) / sim_name
npz = np.load(run_dir / "result_raw.npz", allow_pickle=True)

t = npz["t_ms"]           # (T,)
E = npz["E"]              # (T, N) in Hz
I = npz["I"]              # (T, N) in Hz
labels = npz["region_labels"]  # (N,)

print("Shapes:", t.shape, E.shape, I.shape)
print("Regions:", len(labels))

r = 0  # first region
plt.figure(figsize=(10,3))
plt.plot(t/1000, E[:, r], label="E")
plt.plot(t/1000, I[:, r], label="I")
plt.xlabel("Time (s)"); plt.ylabel("Firing rate (Hz)")
plt.title(f"Region {r} ({labels[r] if labels.size>r else 'R0'})")
plt.legend(); plt.tight_layout(); plt.show()


TypeError: 'module' object is not callable

In [7]:
vars_int = ['E', 'I']
monitor = 'Raw'

for simconfig in simconfigs:
    for seed in seeds:
        result, for_explan = get_result(
            simconfig=simconfig,
            seed=seed,
            vars_int=vars_int,
            time_begin=0.0,          # start at 0 ms for display
            time_end=None,           # up to run_sim
            simu_path=simu_path
        )
        # plot one figure per (config, seed)
        plot_tvb_results(
            simconfigs=[simconfig],
            result=[[result]],       # legacy nested signature: [[result]]
            monitor=monitor,
            for_explan=for_explan,
            var_select=vars_int,
            seeds=[seed],
            figsize=(12,4),
            begin_time=0.0,
            save=0,                  # 0 show, 1 show+save, 2 save only
            save_path=None,
            regions=0,               # plot Region 0; try "mean" or a list like [0,1,2]
            with_title=True
        )


Loading: ./results/synch/b_e_5_E_L_e_-63_0/


IndexError: boolean index did not match indexed array along axis 0; size of axis is 2 but size of corresponding boolean axis is 1